In [1]:
# 경고 메세지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold
from sklearn.model_selection import KFold           # 랜덤하게 섞어서 자를 수도 있고, 순서대로 자를 수도 있음
from sklearn.model_selection import StratifiedKFold # 결과 데이터의 비율이 최대한 균등하게 들어갈 수 있도록!

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 다중분류를 위한 핫-윈 인코더
from keras.utils import to_categorical

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family']='Malgun Gothic'
# 맥용 plt.rcParams['font.family']='AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터 입력
df1 = pd.read_csv('../../data/sonar.csv', header=None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [4]:
# 입력과 결과로 분리한다

x = df1.drop(60, axis=1)
y = df1[60]

display(x)
display(y)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

In [5]:
# 문자열을 숫자로 변환한다

encoder1 = LabelEncoder()
encoder1.fit(y)
y = encoder1.transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
# 학습 데이터와 테스트 데이터로 나눈다
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

In [9]:
# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [10]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# 모델 실행
model.fit(x_train, y_train, epochs=200, batch_size=5)

Epoch 1/200
29/29 [==============================] - 1s 2ms/step - loss: 0.6974 - accuracy: 0.5033
Epoch 2/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6634 - accuracy: 0.5546
Epoch 3/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6575 - accuracy: 0.6228
Epoch 4/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6407 - accuracy: 0.7593
Epoch 5/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6038 - accuracy: 0.7334
Epoch 6/200
29/29 [==============================] - 0s 2ms/step - loss: 0.6105 - accuracy: 0.7227
Epoch 7/200
29/29 [==============================] - 0s 2ms/step - loss: 0.5571 - accuracy: 0.7641
Epoch 8/200
29/29 [==============================] - 0s 2ms/step - loss: 0.5617 - accuracy: 0.6977
Epoch 9/200
29/29 [==============================] - 0s 2ms/step - loss: 0.5408 - accuracy: 0.7541
Epoch 10/200
29/29 [==============================] - 0s 2ms/step - loss: 0.5156 - accuracy: 0.7510
Epoch 11/

29/29 [==============================] - 0s 2ms/step - loss: 0.1860 - accuracy: 0.9463
Epoch 84/200
29/29 [==============================] - 0s 2ms/step - loss: 0.2049 - accuracy: 0.9423
Epoch 85/200
29/29 [==============================] - ETA: 0s - loss: 0.2115 - accuracy: 0.90 - 0s 2ms/step - loss: 0.2098 - accuracy: 0.9076
Epoch 86/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1472 - accuracy: 0.9777
Epoch 87/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1556 - accuracy: 0.9579
Epoch 88/200
29/29 [==============================] - 0s 2ms/step - loss: 0.2182 - accuracy: 0.9269
Epoch 89/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1610 - accuracy: 0.9344
Epoch 90/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1804 - accuracy: 0.9710
Epoch 91/200
29/29 [==============================] - 0s 2ms/step - loss: 0.1979 - accuracy: 0.9380
Epoch 92/200
29/29 [==============================] - 0s 2ms/step - los

29/29 [==============================] - 0s 2ms/step - loss: 0.0381 - accuracy: 1.0000
Epoch 164/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0491 - accuracy: 0.9872
Epoch 165/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0454 - accuracy: 0.9946
Epoch 166/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0421 - accuracy: 1.0000
Epoch 167/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0516 - accuracy: 0.9963
Epoch 168/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 169/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 170/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 171/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 172/200
29/29 [==============================] - 0s 2ms/step - loss: 0.0342 - accuracy: 1.0000
Epoc

In [12]:
# 학습 데이터를 통한 평가
# 학습한 데이터에 대해서는 정확도 100%를 보임

model.evaluate(x_train, y_train)[1]

5/5 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 1.0000


1.0

In [13]:
# 학습하지 않은 데이터를 통한 평가 => 과적합이 일어났음을 알 수 있음

model.evaluate(x_test, y_test)[1]

2/2 [==============================] - 0s 2ms/step - loss: 1.2553 - accuracy: 0.7619


0.761904776096344